In [2]:
!pip install --upgrade --quiet pip
!pip install --quiet python-dotenv
!pip install --quiet langchain langgraph langchain_community
!pip install --quiet transformers mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.2 MB/s eta 0:00:00


In [4]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [5]:
from langchain.agents import create_react_agent
from mistralai import Mistral
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

import os

In [9]:
# save MISTRAL_API_KEY in .env file
api_key = os.getenv("MISTRAL_API_KEY")

In [10]:
client = Mistral(api_key=api_key)

In [11]:
model = "mistral-large-latest"
chat_completion = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)

In [12]:
class Agent:
    def __init__(self, model, system=""):
        self.model = model
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.complete(
                        model=model,
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content


In [13]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [14]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [15]:
abot = Agent(model, prompt)

In [16]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should find out the average weight of a toy poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [17]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [18]:
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs'